In [30]:
%run data_read.py

Data path: C:\Users\Pop\Documents\GitHub\utcc_independent_study\training\..\data\data.xlsx
data_size: 1000
variable: data train_data test_data


In [31]:
data, vocab2index = preprocess_data(data)
data = data[['comment', 'score', 'encoded']]

X = list(data['encoded'])
Y = list(data['score'])
batch_size = 16

train_dl, val_dl, vocab_size, embedding_dim, hidden_dim = create_datasets(X, Y, vocab2index, batch_size)

data.head()

num_words before: 2648
num_words after: 1338
<class 'list'>


,comment,score,encoded
481,ใช้งานได้จริงใช้งานได้จริง,1,"[2, 3, 4, 2, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
134,คุณภาพเสียงนิ่มนวลดีชอบมาก ขนาดกล่องกำลังดีไม่...,6,"[5, 6, 1, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,..."
514,สั่งแบบไร้สายแต่ได้แบบมีสายมาค่ะ แอบเสียใจนิดห...,7,"[40, 36, 41, 18, 3, 36, 42, 43, 44, 45, 10, 46..."
507,สินค้าดี สวย ตรงปก,3,"[52, 7, 10, 53, 10, 54, 55, 0, 0, 0, 0, 0, 0, ..."
775,ได้รับสินค้าแล้ว สภาพสินค้าดีไม่มีบุบไม่มีแตก...,8,"[56, 52, 57, 58, 10, 59, 52, 7, 14, 42, 60, 14..."


In [32]:
def train_model_regr(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        # print('epochs:', i, end=' ')
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.float()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.l1_loss(y_pred, y.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss = validation_metrics_regr(model, val_dl)
        if i % 5 == 1:
            print("train mae %.4f val mae %.4f" % (sum_loss/total, val_loss))
            
def validation_metrics_regr (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.float()
        y_hat = model(x, l)
        loss = F.l1_loss(y_hat, y.unsqueeze(-1))
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total


In [33]:
class LSTM_regr(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.4)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

class RNN_regr(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.4)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        rnn_out, ht = self.rnn(x)
        return self.linear(ht[-1])

class CNN_regr(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, num_filters, filter_sizes, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.convs = nn.ModuleList([
            nn.Conv1d(embedding_dim, num_filters, fs)
            for fs in filter_sizes
        ])
        self.linear1 = nn.Linear(num_filters * len(filter_sizes), hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.4)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = x.permute(0, 2, 1) # switch to (batch_size, embedding_dim, sequence_length)
        conv_outputs = []
        for conv in self.convs:
            conv_output = conv(x)
            conv_output = F.relu(conv_output)
            max_pool_output = F.max_pool1d(conv_output, conv_output.size()[2])
            conv_outputs.append(max_pool_output.squeeze(-1))
        x = torch.cat(conv_outputs, dim=1)
        x = self.dropout(x)
        x = self.linear1(x)
        x = F.relu(x)
        return self.linear2(x)
    
class DNN_regr(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.4)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        x = torch.mean(x, dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        return self.fc3(x)


In [34]:
lstm_model = LSTM_regr(vocab_size, embedding_dim, hidden_dim)
train_model_regr(lstm_model, epochs=20, lr=5e-5)

<class 'numpy.ndarray'> [  14  123   10   18  133  247  752  751 1110  670    1    1  534 1111
  157   10   14 1112  122   10  413  499  751   50    9    1  950   10
 1113   56  141   57 1114  255  451  968   10  583   14  161 1115  968
  367   10  435  111   44  161   10  579  175  237    7   79    1  132
  342    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   

<class 'numpy.ndarray'> [ 56 747 748  57  45  10 141  54  55  64  40 101  10 431 380 329  20 614
 333 146   9  33  45  10  65  66  96  70  71  39  26  45   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
<class 'numpy.ndarray'> [317 411 718 317 411  18  97  10 408  10 111 368  10 873 720  14   1 105
  10 366  18 112  10  38  78 304  31 703  14  83  79 212  10  47  

<class 'numpy.ndarray'> [  52  199   54   55  643   10   18  357  146  394   10  535  111   10
   28  283  656   14  366  643   10    1   14  219   10   14  338  144
  176  121 1175    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   

<class 'numpy.ndarray'> [  37  207  689   10   14    1   10  399  714  656   19  113   10  862
  953   10  170  171 1123   10  877   26   10  740   92  993   10   96
 1121   10   57 1123   57  846   14  953    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   

<class 'numpy.ndarray'> [ 56  52  57  45  10  18 107  14   3 133 146 144 491   7  11 148  10  48
  14  42 277  33  10 170 133 370 112 132 648  77   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
<class 'numpy.ndarray'> [  84   49    9   53   54   55  167 1201   10   18  613  113  175   79
  358  101  167 1201    0    0    0    0    0    0    0    0    0   

...

<class 'numpy.ndarray'> [ 404    1   14  113   10   73  161  339  340  358   10 1205   14  338
   38   32  343    3  671   10 1107   31   14  161    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   

<class 'numpy.ndarray'> [112  14 113  45   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
<class 'numpy.ndarray'> [  49  297  354   10   31    4   10   18   73  134   10   14    7  428
   10  202   14  169   10   18   10  365   73  134  402   10  810  7

<class 'numpy.ndarray'> [ 79 358   9  45  10 170 132  62 141 435 534   9   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
<class 'numpy.ndarray'> [  14    7    1  358   10  701   14    7  142   96  846 1127  946   10
  572  195    9   10   73  134   19    7   10  701   10   38   97  1

<class 'numpy.ndarray'> [  56   52   57  250   78    7    3  135  296  844  475    6  406  294
 1134  107   14    3 1132  738  247   44   20  991  717  106  367   67
  337   80   52    7  114 1135    3  135  296  844  475  821 1135    3
  294 1134  107   14    3 1132  970  247   44   20  991  717  106  367
   69   67 1058  822  599  615    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   

KeyboardInterrupt: 

In [23]:
rnn_model = RNN_regr(vocab_size, embedding_dim, hidden_dim)
train_model_regr(rnn_model, epochs=20, lr=5e-5)

train mae 3.5676 val mae 2.6525


KeyboardInterrupt: 

In [9]:
num_filters = 100
filter_sizes = [3, 4, 5]
cnn_model = CNN_regr(vocab_size, embedding_dim, num_filters, filter_sizes, hidden_dim)
train_model_regr(cnn_model, epochs=20, lr=5e-5)

train mae 4.0723 val mae 3.6759
train mae 1.2372 val mae 1.0450
train mae 1.0450 val mae 0.9666
train mae 0.8474 val mae 0.9335


In [10]:
dnn_model = DNN_regr(vocab_size, embedding_dim, hidden_dim)
train_model_regr(dnn_model, epochs=20, lr=5e-5)

train mae 4.8941 val mae 5.0823
train mae 4.7682 val mae 4.9340
train mae 4.4411 val mae 4.5771
train mae 3.6692 val mae 3.6969


In [ ]:
model = torch.load(checkpoint_path)
model.eval()

# Read the test excerpts
test_data = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
print(test_data.head())

# Apply the same encoding as the train texts
test_data['encoded'] = test_data['excerpt'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
idx, excerpts_test = test_data['id'], test_data['encoded']

X_test = [excerpts_test[i][0] for i in range(len(test_data))]
l_test = [excerpts_test[i][1] for i in range(len(test_data))]
X_test = torch.LongTensor(X_test)